In [ ]:
import pyvisa
from time import perf_counter_ns, sleep
import numpy as np
import plotly.express as px
import pandas as pd
from typing import List, Dict, TypedDict


class RigolPSU:
    def __init__(self, resource_index=0):
        rm = pyvisa.ResourceManager()
        resources = rm.list_resources()
        if not resources:
            raise RuntimeError("No VISA instruments found.")
        self.instrument = rm.open_resource(resources[resource_index])

        # Set sensible defaults
        self.instrument.timeout = 5000
        self.instrument.read_termination = '\n'
        self.instrument.write_termination = '\n'

        self.on = False
        self.channel = 1

        print("Connected to:", self.instrument.query("*IDN?").strip())

    def set_voltage(self, voltage: float, current_limit: float):
        self.instrument.write(f"APPL CH{self.channel},{voltage},{current_limit}")
        if not self.on:
            self.instrument.write(f"OUTP CH{self.channel},ON")
            self.on = True

    def measure_power(self) -> float:
        voltage = float(self.instrument.query(f"MEAS:VOLT? CH{self.channel}"))
        current = float(self.instrument.query(f"MEAS:CURR? CH{self.channel}"))
        return voltage * current
    
    def measure_power_over_time(self, seconds: float, time_step=.05):
        start = perf_counter_ns()
        end = start + seconds * 1e9
        time = []
        power = []
        while (current_time := perf_counter_ns()) < end:
            time.append(current_time)
            power.append(self.measure_power())
            sleep(time_step)
        return dict(x=(np.array(time)-start)/1e9, y=np.array(power))
    
    def volt_sweep(self, total_time=10.0, max_voltage=10.0, current_limit: float = 2.0, measure_time=1):

        power_measurements = []
        voltages = np.linspace(0, max_voltage, round(total_time/measure_time)).round(1)
        for voltage in voltages:
            self.set_voltage(voltage, current_limit)
            sleep(measure_time)
            power_measurements.append(round(self.measure_power(), 2))

        return dict(x=voltages, y=np.array(power_measurements))
    
    def output_on(self):
        self.instrument.write(f"OUTP CH{self.channel},ON")
        self.on = True

    def output_off(self):
        self.instrument.write(f"OUTP CH{self.channel},OFF")
        self.on = False

    def close(self):
        if self.on:
            self.output_off()
        self.instrument.close()

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.close()




In [ ]:
with RigolPSU() as psu:
    # psu.set_voltage(voltage=5.0, current_limit=1.0)
    px.line(**psu.volt_sweep()).show()